# Multiple URL Content Extraction with WebBaseLoader

여러 기술 블로그 사이트들로부터 동시에 콘텐츠를 추출하고 분석하는 테스트

In [1]:
# 필요한 패키지 import
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import time
import os
from dotenv import load_dotenv

# Environment variables 로드
load_dotenv()

# OpenAI API 키 확인
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY가 .env 파일에 설정되지 않았습니다.")

print("✅ 모든 패키지 import 완료")

USER_AGENT environment variable not set, consider setting it to identify your requests.


✅ 모든 패키지 import 완료


In [2]:
# 테스트할 기술 블로그 URL 목록
urls = [
    "http://thefarmersfront.github.io/blog/2025-delivery-jarvis-story/",  
    "https://medium.com/musinsa-tech/무진장을-맞아-후기-응답속도를-개선해보자-c92e0ae60f1e",
    "https://medium.com/daangn/ai-툴-개발은-처음이라-당근-비개발자-구성원들의-ai-도전기-fb62d2a6c2f3",
    "https://tech.socarcorp.kr/fe/2025/06/10/monorepo-ci-cd-pipeline.html",
    "https://d2.naver.com/news/6518915",
    "https://meetup.nhncloud.com/posts/394",
    "https://tech.kakao.com/posts/724",
    "https://techblog.lycorp.co.jp/ko/extracting-trending-keywords-from-openchat-messages",
    "https://developers.kakaomobility.com/docs/techblogs/address-structure-1/",
    "https://oliveyoung.tech/2025-09-08/gms-qa-strategy/",
    "https://toss.tech/article/tosspeople_hyunjung",
    "https://techblog.gccompany.co.kr/%EC%97%AC%EA%B8%B0%EC%96%B4%EB%95%8C-ci-cd-%EA%B0%9C%EC%84%A0%EA%B8%B0-part-5-slack%EC%9C%BC%EB%A1%9C-%EC%99%84%EC%84%B1%EB%90%98%EB%8A%94-%EB%B0%B0%ED%8F%AC-%EA%B0%80%EC%8B%9C%EC%84%B1-c38215b4ed61?source=rss----18356045d353---4",
    "https://techblog.woowahan.com/22767/",
]

print(f"📋 총 {len(urls)}개의 기술 블로그 URL 준비 완료")
print("\n🎯 테스트 대상 사이트들:")
for i, url in enumerate(urls, 1):
    print(f"   {i:2d}. {url}")

📋 총 13개의 기술 블로그 URL 준비 완료

🎯 테스트 대상 사이트들:
    1. http://thefarmersfront.github.io/blog/2025-delivery-jarvis-story/
    2. https://medium.com/musinsa-tech/무진장을-맞아-후기-응답속도를-개선해보자-c92e0ae60f1e
    3. https://medium.com/daangn/ai-툴-개발은-처음이라-당근-비개발자-구성원들의-ai-도전기-fb62d2a6c2f3
    4. https://tech.socarcorp.kr/fe/2025/06/10/monorepo-ci-cd-pipeline.html
    5. https://d2.naver.com/news/6518915
    6. https://meetup.nhncloud.com/posts/394
    7. https://tech.kakao.com/posts/724
    8. https://techblog.lycorp.co.jp/ko/extracting-trending-keywords-from-openchat-messages
    9. https://developers.kakaomobility.com/docs/techblogs/address-structure-1/
   10. https://oliveyoung.tech/2025-09-08/gms-qa-strategy/
   11. https://toss.tech/article/tosspeople_hyunjung
   12. https://techblog.gccompany.co.kr/%EC%97%AC%EA%B8%B0%EC%96%B4%EB%95%8C-ci-cd-%EA%B0%9C%EC%84%A0%EA%B8%B0-part-5-slack%EC%9C%BC%EB%A1%9C-%EC%99%84%EC%84%B1%EB%90%98%EB%8A%94-%EB%B0%B0%ED%8F%AC-%EA%B0%80%EC%8B%9C%EC%84%B1-c38215b4ed61?sour

In [4]:
# WebBaseLoader 설정 및 초기화
def setup_webbase_loader(urls_list, requests_per_second=2):
    """
    WebBaseLoader를 설정하고 초기화하는 함수
    """
    loader = WebBaseLoader(urls_list)
    
    # 요청 설정 - test0925.py 참고
    loader.requests_kwargs = {
        'verify': False,  # SSL 검증 비활성화
        'timeout': 30,    # 타임아웃 설정
    }
    
    # 동시 요청 속도 제어
    loader.requests_per_second = requests_per_second
    
    return loader

# 텍스트 분할기 설정 - test0925.py 참고
text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=3000,     # 쪼개는 글자수
    chunk_overlap=300,   # 오버랩 글자수
    length_function=len,
    is_separator_regex=False,
)

print("✅ WebBaseLoader 및 텍스트 분할기 설정 완료")

✅ WebBaseLoader 및 텍스트 분할기 설정 완료


In [5]:
# 방법 1: 모든 URL을 한 번에 처리하기
print("🚀 방법 1: 모든 URL을 한 번에 처리하기")
print("-" * 60)

start_time = time.time()

try:
    # 모든 URL을 한 번에 로드
    loader = setup_webbase_loader(urls, requests_per_second=3)
    docs = loader.load_and_split(text_splitter)
    
    end_time = time.time()
    processing_time = end_time - start_time
    
    print(f"✅ 성공: {len(docs)}개 청크 추출 완료")
    print(f"⏱️  처리 시간: {processing_time:.2f}초")
    print(f"📊 평균 처리 시간: {processing_time/len(urls):.2f}초/URL")
    
except Exception as e:
    print(f"❌ 오류 발생: {e}")
    docs = []

🚀 방법 1: 모든 URL을 한 번에 처리하기
------------------------------------------------------------


/Users/apple/IdeaProjects/techbloghub/python/venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'helloworld.kurly.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/apple/IdeaProjects/techbloghub/python/venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'medium.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/apple/IdeaProjects/techbloghub/python/venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'medium.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advan

✅ 성공: 28개 청크 추출 완료
⏱️  처리 시간: 9.51초
📊 평균 처리 시간: 0.73초/URL


In [6]:
# 방법 2: 개별 URL 처리하기 (실패 시 대안)
print("\n🔄 방법 2: 개별 URL 처리하기")
print("-" * 60)

individual_docs = []
successful_urls = []
failed_urls = []

start_time = time.time()

for i, url in enumerate(urls, 1):
    try:
        print(f"📥 {i:2d}/{len(urls)} 처리 중: {url}")
        
        # 개별 URL 로더 생성
        single_loader = setup_webbase_loader([url], requests_per_second=1)
        single_docs = single_loader.load_and_split(text_splitter)
        
        individual_docs.extend(single_docs)
        successful_urls.append(url)
        
        print(f"   ✅ 성공: {len(single_docs)}개 청크 추출")
        
    except Exception as e:
        failed_urls.append(url)
        print(f"   ❌ 실패: {str(e)[:100]}...")
    
    # 과도한 요청 방지를 위한 딜레이
    time.sleep(1)

end_time = time.time()
processing_time = end_time - start_time

print(f"\n📊 개별 처리 결과:")
print(f"   ✅ 성공: {len(successful_urls)}개 URL")
print(f"   ❌ 실패: {len(failed_urls)}개 URL")
print(f"   📄 총 청크: {len(individual_docs)}개")
print(f"   ⏱️  총 처리 시간: {processing_time:.2f}초")


🔄 방법 2: 개별 URL 처리하기
------------------------------------------------------------
📥  1/13 처리 중: http://thefarmersfront.github.io/blog/2025-delivery-jarvis-story/
   ✅ 성공: 2개 청크 추출


/Users/apple/IdeaProjects/techbloghub/python/venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'helloworld.kurly.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


📥  2/13 처리 중: https://medium.com/musinsa-tech/무진장을-맞아-후기-응답속도를-개선해보자-c92e0ae60f1e


/Users/apple/IdeaProjects/techbloghub/python/venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'medium.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
Created a chunk of size 6177, which is longer than the specified 3000


   ✅ 성공: 1개 청크 추출


...

📥 12/13 처리 중: https://techblog.gccompany.co.kr/%EC%97%AC%EA%B8%B0%EC%96%B4%EB%95%8C-ci-cd-%EA%B0%9C%EC%84%A0%EA%B8%B0-part-5-slack%EC%9C%BC%EB%A1%9C-%EC%99%84%EC%84%B1%EB%90%98%EB%8A%94-%EB%B0%B0%ED%8F%AC-%EA%B0%80%EC%8B%9C%EC%84%B1-c38215b4ed61?source=rss----18356045d353---4


/Users/apple/IdeaProjects/techbloghub/python/venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'techblog.gccompany.co.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/apple/IdeaProjects/techbloghub/python/venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'medium.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/apple/IdeaProjects/techbloghub/python/venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'techblog.gccompany.co.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.

   ✅ 성공: 1개 청크 추출
📥 13/13 처리 중: https://techblog.woowahan.com/22767/


/Users/apple/IdeaProjects/techbloghub/python/venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'techblog.woowahan.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


   ✅ 성공: 6개 청크 추출

📊 개별 처리 결과:
   ✅ 성공: 13개 URL
   ❌ 실패: 0개 URL
   📄 총 청크: 28개
   ⏱️  총 처리 시간: 19.12초


In [7]:
# 성공한 결과 분석
final_docs = docs if docs else individual_docs

if final_docs:
    print("\n📈 콘텐츠 추출 결과 분석")
    print("=" * 80)
    
    # 사이트별 통계
    site_stats = {}
    for doc in final_docs:
        source = doc.metadata.get('source', 'Unknown')
        if source not in site_stats:
            site_stats[source] = {
                'chunks': 0,
                'total_length': 0,
                'title': doc.metadata.get('title', '제목 없음')[:50]
            }
        site_stats[source]['chunks'] += 1
        site_stats[source]['total_length'] += len(doc.page_content)
    
    for i, (url, stats) in enumerate(site_stats.items(), 1):
        print(f"\n🌐 사이트 {i}: {url}")
        print(f"   📝 제목: {stats['title']}")
        print(f"   📊 청크 수: {stats['chunks']}개")
        print(f"   📏 총 길이: {stats['total_length']:,}자")
        print(f"   📄 평균 청크 크기: {stats['total_length']//stats['chunks']:,}자")
        
        # 첫 번째 청크의 미리보기
        first_chunk = next((doc for doc in final_docs if doc.metadata.get('source') == url), None)
        if first_chunk:
            preview = first_chunk.page_content[:200].replace('\n', ' ').strip()
            print(f"   👀 미리보기: {preview}...")
        
        print("-" * 60)
else:
    print("\n❌ 추출된 콘텐츠가 없습니다.")


📈 콘텐츠 추출 결과 분석

🌐 사이트 1: http://thefarmersfront.github.io/blog/2025-delivery-jarvis-story/
   📝 제목: 우리 팀에도 Jarvis 가 생겼다 – 생성형 AI 로 만든 에러 분석가 이야기 - 컬리 
   📊 청크 수: 2개
   📏 총 길이: 5,581자
   📄 평균 청크 크기: 2,790자
   👀 미리보기: 우리 팀에도 Jarvis 가 생겼다 – 생성형 AI 로 만든 에러 분석가 이야기 - 컬리 기술 블로그   Kurly Tech Blog  우리 팀에도 Jarvis 가 생겼다 – 생성형 AI 로 만든 에러 분석가 이야기  팀 내에서 생성형 AI 를 활용한 사례를 공유합니다.  김성준 게시 날짜: 2025.07.01.  왜 에러 로그는 항상 늦게 처리되는가? 에...
------------------------------------------------------------

🌐 사이트 2: https://medium.com/musinsa-tech/무진장을-맞아-후기-응답속도를-개선해보자-c92e0ae60f1e
   📝 제목: 무진장을 맞아, 후기 응답속도를 개선해보자. 안녕하세요. 무신사 구매개발실 회원개발팀에서 
   📊 청크 수: 1개
   📏 총 길이: 6,177자
   📄 평균 청크 크기: 6,177자
   👀 미리보기: 무진장을 맞아, 후기 응답속도를 개선해보자. 안녕하세요. 무신사 구매개발실 회원개발팀에서 백엔드 엔지니어로 일하고… | by Soyoun Kim | MUSINSA tech | MediumSitemapOpen in appSign upSign inMedium LogoWriteSign upSign inMUSINSA tech·팀 무신사의 테크 이야기를 공유합니다....
------------------------------------------------------------

🌐 사이트 3: https://medium.com/daangn/ai-툴-개발은-처음이라-

In [8]:
# 실패한 URL들 상세 분석
if failed_urls:
    print("\n⚠️  실패한 URL 분석")
    print("=" * 50)
    
    for i, url in enumerate(failed_urls, 1):
        print(f"{i}. {url}")
        
        # 간단한 재시도
        try:
            retry_loader = WebBaseLoader([url])
            retry_loader.requests_kwargs = {'verify': False, 'timeout': 10}
            retry_docs = retry_loader.load()
            if retry_docs:
                print(f"   ✅ 재시도 성공: {len(retry_docs[0].page_content)} 문자")
            else:
                print(f"   ❌ 재시도 실패: 빈 콘텐츠")
        except Exception as retry_error:
            print(f"   ❌ 재시도 실패: {str(retry_error)[:50]}...")

In [13]:
# 각 URL별로 개별 요약 생성
if final_docs:
    print("\\n🤖 URL별 개별 AI 요약 생성")
    print("=" * 80)
    
    # 템플릿 설정 - test0925.py 참고
    template = '''다음의 내용을 한글로 요약해줘:\\n\\n{text}'''
    
    combine_template = '''{text}\\n\\n요약의 결과는 다음의 형식으로 작성해줘:
제목: 글의 제목
주요내용: 한 줄로 요약된 내용
작성자: 추정된 작성자 또는 블로그명
내용: 주요내용을 불렛포인트 형식으로 작성
'''
    
    # 템플릿 생성
    prompt = PromptTemplate(template=template, input_variables=['text'])
    combine_prompt = PromptTemplate(template=combine_template, input_variables=['text'])
    
    # LLM 객체 생성
    llm = ChatOpenAI(temperature=0, model_name='gpt-4o-mini')
    
    # 요약 체인 생성
    chain = load_summarize_chain(
        llm,
        map_prompt=prompt,
        combine_prompt=combine_prompt,
        chain_type="map_reduce",
        verbose=False
    )
    
    # 각 URL별로 문서들을 그룹화하고 개별 요약 생성
    url_groups = {}
    for doc in final_docs:
        source = doc.metadata.get('source', 'Unknown')
        if source not in url_groups:
            url_groups[source] = []
        url_groups[source].append(doc)
    
    print(f"📊 총 {len(url_groups)}개 사이트의 콘텐츠를 개별 요약합니다...")
    
    summaries = {}
    
    for i, (url, docs_for_url) in enumerate(url_groups.items(), 1):
        try:
            print(f"🔄 {i}/{len(url_groups)} 요약 생성 중...")
            print(f"📍 URL: {url}")
            print(f"📄 청크 수: {len(docs_for_url)}개")
            
            # 요약 실행
            summary = chain.invoke({'input_documents': docs_for_url})
            summaries[url] = summary['output_text']
            
            print(f"✅ 요약 완료")
            print("📋 요약 결과:")
            print("-" * 60)
            print(summary['output_text'])
            print("=" * 80)
            print()
            
        except Exception as e:
            print(f"❌ 요약 생성 실패: {e}")
            print("-" * 60)
            summaries[url] = f"요약 생성 실패: {e}"
            
        # API 호출 간격 조절
        time.sleep(2)
    
    print(f"🎯 전체 요약 완료: {len(summaries)}개 사이트")
    
else:
    print("❌ 요약할 문서가 없습니다.")

\n🤖 URL별 개별 AI 요약 생성
📊 총 13개 사이트의 콘텐츠를 개별 요약합니다...
🔄 1/13 요약 생성 중...
📍 URL: http://thefarmersfront.github.io/blog/2025-delivery-jarvis-story/
📄 청크 수: 2개
✅ 요약 완료
📋 요약 결과:
------------------------------------------------------------
제목: 생성형 AI를 활용한 에러 분석가 'Jarvis'의 도입과 발전 과정  
주요내용: Jarvis는 에러 로그 분석을 자동화하여 개발자들이 신속하게 문제를 해결할 수 있도록 돕는 AI 시스템이다.  
작성자: 김성준 (컬리 기술 블로그)  
내용:  
- 에러 로그는 개발자들에게 복잡하고 시간이 많이 소요되는 문제로, Jarvis를 통해 해결하고자 함.  
- 초기 AI 응답 품질이 낮았으나, 프롬프트 재구성을 통해 개선됨.  
- 팀원들이 Jarvis의 제안을 잘 참고하지 않는 문제를 발견하고, 정보 부족이 원인임을 인식.  
- MCP(Model Context Protocol)를 도입하여 Jarvis가 더 풍부한 맥락을 이해하도록 개선.  
- Jarvis는 실제 운영 환경에서 빠르게 문제를 분석하고 해결 방안을 제시함.  
- 향후 팀원들의 Slack 대화 내용을 학습하여 더욱 깊이 있는 해결책을 제시할 계획.  
- 개발 과정에서의 문제 해결 경험과 팀원들의 협력이 중요하다는 점 강조.  

🔄 2/13 요약 생성 중...
📍 URL: https://medium.com/musinsa-tech/무진장을-맞아-후기-응답속도를-개선해보자-c92e0ae60f1e
📄 청크 수: 1개
✅ 요약 완료
📋 요약 결과:
------------------------------------------------------------
제목: 무신사 후기 서비스 성능 개선 사례  
주요내용: 무신사 후기 서비스의 응답 속도를 개선하여 이벤트 대비 안정성을 확보했다. 

In [10]:
# 최종 결과 요약
print("\n🎯 테스트 완료 - 최종 결과")
print("=" * 60)
print(f"📋 총 테스트 URL: {len(urls)}개")
print(f"✅ 성공한 URL: {len(successful_urls) if 'successful_urls' in locals() else len(site_stats)}개")
print(f"❌ 실패한 URL: {len(failed_urls) if 'failed_urls' in locals() else 0}개")
print(f"📄 총 추출된 청크: {len(final_docs)}개")
print(f"📏 총 콘텐츠 길이: {sum(len(doc.page_content) for doc in final_docs):,}자")

if final_docs:
    avg_chunk_size = sum(len(doc.page_content) for doc in final_docs) // len(final_docs)
    print(f"📊 평균 청크 크기: {avg_chunk_size:,}자")
    
print("\n✨ WebBaseLoader 다중 URL 처리 테스트 완료!")


🎯 테스트 완료 - 최종 결과
📋 총 테스트 URL: 13개
✅ 성공한 URL: 13개
❌ 실패한 URL: 0개
📄 총 추출된 청크: 28개
📏 총 콘텐츠 길이: 80,012자
📊 평균 청크 크기: 2,857자

✨ WebBaseLoader 다중 URL 처리 테스트 완료!
